In [1]:
#%pip install transformers[torch] datasets evaluate mlflow| bo|to3

In [2]:
#%pip install fsspec==2021.10 s3fs==2021.10
#%pip install --upgrade fsspec
#!python3 -m pip install --upgrade pip
#%pip install --upgrade urllib3
#%pip install --upgrade boto3

In [3]:
import os

os.environ["MLFLOW_EXPERIMENT_NAME"] = "LingBizkit_experiment"
os.environ["MLFLOW_TRACKING_URI"] = "http://79.137.194.156:5000/"

In [4]:
from pathlib import Path

CONTENT_PATH = Path(".")

In [5]:
from transformers import set_seed

set_seed(228)

2023-11-24 22:33:57.550850: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
import pandas as pd

dataset_df = pd.read_csv("preproc_dataset.csv")

dataset_df['text'] = dataset_df['text'].apply(lambda x: x[:768])
dataset_df

,executor,theme_group,text,theme,sample_type
0,Министерство здравоохранения,Коронавирус,"Будьте так любезны, отчитайтесь за целевое исп...",Коронавирус,train
1,Министерство социального развития ПК,Мобилизация,Добрый день. Мне подсказали что семьям участни...,Поддержка семей мобилизованных,train
2,Министерство здравоохранения,Здравоохранение/Медицина,Доброй день! Ответьте пожалуйста планируется л...,Предложения по созданию лечебных центров,train
3,Губахинский городской округ,ЖКХ,Суворова 7 второй день без воды,Отсутствие холодной воды,train
4,Министерство социального развития ПК,Социальное обслуживание и защита,Добрый вечер. С октября по декабрь 2021г. Сост...,Занятость и трудоустройство,train
...,...,...,...,...,...
22628,Министерство социального развития ПК,Газ и топливо,да конечно только чтоб подвести газ к дому над...,"Стоимость, оплата и возврат средств на газифик...",test
22629,Министерство здравоохранения,Коронавирус,Почему не тестов в Кичанрва?? Или там нет люде...,Тесты на коронавирус,test
22630,ИГЖН ПК,ЖКХ,Только смирились жители Чусового с тарифами на...,Завышение платы за коммунальные услуги,test
22631,Город Пермь,Благоустройство,Добрый день! В Закамске на ост.Рубин раньше бы...,Предложение по благоустройству,test


In [11]:
dataset_df.groupby('executor')['text'].count()

executor
АО ПРО ТКО                                            1479
Александровский муниципальный округ Пермского края     960
Бардымский муниципальный округ Пермского края          372
Город Пермь                                           4430
Губахинский городской округ                           1076
ИГЖН ПК                                               1808
Лысьвенский городской округ                           2029
Министерство здравоохранения                          5695
Министерство образования                               506
Министерство социального развития ПК                  4278
Name: text, dtype: int64

In [7]:
id2label = {k: v for k, v in enumerate(dataset_df.executor.unique())}
label2id = {v: k for k, v in id2label.items()}
len(label2id)

10

In [12]:
from datasets import Dataset

temp_dataset_df = dataset_df.rename({"executor": "labels"}, axis=1)
temp_dataset_df.labels = temp_dataset_df.labels.map(lambda x: label2id[x])
train_dataset = Dataset.from_pandas(
    temp_dataset_df[temp_dataset_df.sample_type == "train"][["text", "labels"]]
)
test_dataset = Dataset.from_pandas(
    temp_dataset_df[temp_dataset_df.sample_type == "test"][["text", "labels"]]
)

In [13]:
from transformers import AutoTokenizer

model_name = 'cointegrated/LaBSE-en-ru'
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/521k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [14]:
def preprocess_function(examples):
    return tokenizer(examples["text"], return_tensors='pt', truncation=True, padding=True)

In [15]:
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)

Parameter 'function'=<function preprocess_function at 0x7f54effb93f0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/18119 [00:00<?, ? examples/s]

Map:   0%|          | 0/4514 [00:00<?, ? examples/s]

In [16]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [17]:
import evaluate

metric = evaluate.load("f1")

In [18]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels, average='weighted')

In [19]:
len(id2label)

10

In [20]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(id2label))
model.to('cuda')

model.safetensors:   0%|          | 0.00/516M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/LaBSE-en-ru and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(55083, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir=model_name+"_executor",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy="steps",
    evaluation_strategy="steps",
    eval_accumulation_steps=5, 
    load_best_model_at_end=True,
    save_total_limit=2, 
    report_to="none"
)



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,F1
500,0.871000,0.691264,0.761671
1000,0.575000,0.634274,0.793883


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,F1
500,1.061900,1.815970,0.520003
1000,0.879100,1.857754,0.529818
1500,0.730500,1.951232,0.522273
2000,0.620400,1.920849,0.534887


In [20]:
# import mlflow
# from transformers import TrainingArguments, Trainer

# training_args = TrainingArguments(
#     output_dir=model_name+"_test",
#     overwrite_output_dir = 'True',
#     learning_rate=2e-5,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     num_train_epochs=10,
#     weight_decay=0.01,
#     evaluation_strategy="epoch",
#     #eval_accumulation_steps=10, 
#     save_strategy="epoch",
#     save_total_limit = 2,
    
#     load_best_model_at_end=True,
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_train,
#     eval_dataset=tokenized_test,
#     tokenizer=tokenizer,
#     data_collator=data_collator,
#     compute_metrics=compute_metrics,
# )

# trainer.train()
# trainer.save_model(f"model_best")

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


AttributeError: 'NoneType' object has no attribute 'get'

Fri Nov 24 01:19:56 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.147.05   Driver Version: 525.147.05   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:8C:00.0 Off |                    0 |
| N/A   39C    P0    36W / 250W |   5722MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
trainer??

Type:        Trainer
String form: <transformers.trainer.Trainer object at 0x7f5d9e20efb0>
File:        ~/.local/lib/python3.10/site-packages/transformers/trainer.py
Source:     
class Trainer:
    """
    Trainer is a simple but feature-complete training and eval loop for PyTorch, optimized for 🤗 Transformers.

    Args:
        model ([`PreTrainedModel`] or `torch.nn.Module`, *optional*):
            The model to train, evaluate or use for predictions. If not provided, a `model_init` must be passed.

            <Tip>

            [`Trainer`] is optimized to work with the [`PreTrainedModel`] provided by the library. You can still use
            your own models defined as `torch.nn.Module` as long as they work the same way as the 🤗 Transformers
            models.

            </Tip>

        args ([`TrainingArguments`], *optional*):
            The arguments to tweak for training. Will default to a basic instance of [`TrainingArguments`] with the
            `output_dir` set to a dir

In [ ]:
!nvidia-smi

Thu Nov 23 22:52:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.147.05   Driver Version: 525.147.05   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:8C:00.0 Off |                    0 |
| N/A   29C    P0    26W / 250W |      4MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!df -h

Filesystem                           Size  Used Avail Use% Mounted on
overlay                               99G   69G   27G  73% /
tmpfs                                 64M     0   64M   0% /dev
tmpfs                                 48G     0   48G   0% /sys/fs/cgroup
/dev/vda2                             99G   69G   27G  73% /servant
overlay                               99G   69G   27G  73% /kernel
[2a0d:d6c1:1::1:6e]:/                 10G  9.6G     0 100% /home/jupyter
[2a0d:d6c1:1::1:6e]:/mnt              10G  9.6G     0 100% /home/jupyter/mnt
[2a0d:d6c1:1::1:6e]:/mnt/s3           10G  9.6G     0 100% /home/jupyter/mnt/s3
[2a0d:d6c1:1::1:6e]:/work/resources   10G  9.6G     0 100% /home/jupyter/work/resources
tmpfs                                 48G  4.0K   48G   1% /dev/shm
tmpfs                                 48G   12K   48G   1% /proc/driver/nvidia
tmpfs                                9.5G  1.6M  9.5G   1% /run/nvidia-persistenced/socket


In [ ]:
import boto3
import os
from os import path


In [5]:
import shutil

In [4]:
shutil.rmtree('/home/jupyter/datasphere/project/Untitled Folder/rubert-tiny2')

In [6]:
%ls ~/.cache/huggingface/hub

ls: cannot access '/home/jupyter/.cache/huggingface/hub': No such file or directory


Exception: Process exited with code 2

In [ ]:
pd.read_csv